In [2]:
import pandas as pd
import os

### Parsing conversations

In [73]:
all_columns=['conversation_id', 'id','author_id','text','in_reply_to_user_id']
for file in os.listdir("conversations"):
    if file.split(".")[1]=="json":
        print("--------------Processing file {0}--------------".format(file))
        df=pd.read_json(os.path.join("conversations",file),lines=True)
        all_tweets=[]
        i=100
        for line in df['data']:
            for tweet in line:
                tweet_info=[]
                for column in all_columns:
                    tweet_info.append(tweet[column])
                try:
                    if tweet['referenced_tweets'][0]['type']=='replied_to':
                        tweet_info.append(tweet['referenced_tweets'][0]['id'])
                    else:
                        tweet_info.append('')
                except:
                    print(tweet['referenced_tweets'][0])
                    tweet_info.append('')
                all_tweets.append(tweet_info)
            if len(all_tweets)>i:
                print("--------------Processed more than {0} tweets--------------".format(i))
                i=i*10

        for line in df['includes']:
            for tweet in line['tweets']:
                tweet_info=[]
                for column in all_columns:
                    tweet_info.append(tweet.get(column,''))
                if tweet.get('referenced_tweets','')!='':
                    if tweet['referenced_tweets'][0]['type']=='replied_to':
                        tweet_info.append(tweet['referenced_tweets'][0]['id'])
                    else:
                        tweet_info.append('')
                else:
                    tweet_info.append('')
                all_tweets.append(tweet_info)
            if len(all_tweets)>i:
                print("--------------Processed more than {0} tweets--------------".format(i))
                i=i*10
        df_out = pd.DataFrame(all_tweets,columns=all_columns+['replied_to'])
        df_out = df_out.drop_duplicates().copy()
        df_out.to_csv("Parsed_conversations/"+file.split(".")[0]+".csv", index=False)

--------------Processing file Australia_ids.json--------------
--------------Processed more than 100 tweets--------------
--------------Processed more than 1000 tweets--------------
--------------Processing file Britain_ids.json--------------
--------------Processed more than 100 tweets--------------
--------------Processed more than 1000 tweets--------------
--------------Processed more than 10000 tweets--------------
--------------Processing file Canada_ids.json--------------
--------------Processed more than 100 tweets--------------
--------------Processed more than 1000 tweets--------------
--------------Processed more than 10000 tweets--------------
--------------Processing file ICRC_library.json--------------
--------------Processed more than 100 tweets--------------
--------------Processed more than 1000 tweets--------------
--------------Processing file IFRC_Europe.json--------------
--------------Processed more than 100 tweets--------------
--------------Processed more than 10

### Concating old and new files

In [89]:
ivan_dict={"nepal_ids":"NepalRedCross","Nairobi_ids":"ICRC_Nairobi","Canada_ids":"redcrosscanada","Britain_ids":"BritishRedCross","Australia_ids":"RedCrossAU","Ireland_ids":"irishredcross"}

In [90]:
NGO_list = ["Australia_ids", "Britain_ids", "Canada_ids", "Ireland_ids", "Nairobi_ids", "nepal_ids", "RedCrossLebanon", "RedCross", "philredcross", "IFRC_Europe", "ICRC_library"]
files=os.listdir("parsed_conversations")
for ngo in NGO_list:
    group=[file for file in files if ngo in file]
    if 'RedCross.csv' in group:
        group=[element for element in group if 'Lebanon' not in element]
    for ind,element in enumerate(group):
        if ind==0:
            df=pd.read_csv("parsed_conversations/"+element)
        else:
            df1=pd.read_csv("parsed_conversations/"+element)
            df=pd.concat([df,df1]).copy()
    df.to_csv("parsed_conversations/"+ivan_dict.get(ngo,ngo)+"_all.csv",index=False)
        